The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [17]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [18]:
from crewai import Agent, Task, Crew

In [19]:
import os
from dotenv import load_dotenv
load_dotenv() 

print(os.environ["OPENAI_MODEL_NAME"], " at ", os.environ["OPENAI_API_BASE"])

gpt-4.1-nano  at  https://api.proxyapi.ru/openai/v1


In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4.1-nano", temperature=0.1)

In [21]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
    llm=llm,
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [22]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
    llm=llm,
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [23]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [24]:
docs_scrape_tool = ScrapeWebsiteTool(website_url='https://docs.crewai.com/concepts/memory'
    #website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [25]:
docs_scrape_tool.run()

Using Tool: Read website content


'Memory - CrewAI CrewAI home page Search CrewAI docs Start Free Trial crewAIInc / crewAI crewAIInc / crewAI Search... Navigation Core Concepts Memory Documentation Enterprise Examples Releases Website Forum Get Help Get Started Introduction Installation Quickstart Guides Strategy Agents Crews Flows Advanced Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes Collaboration Training Memory Planning Testing CLI Tools Event Listeners Tools AI Mind Tool Apify Actors Bedrock Invoke Agent Tool Bedrock Knowledge Base Retriever Brave Search Browserbase Web Loader Code Docs RAG Search Code Interpreter Composio Tool CSV RAG Search DALL-E Tool Directory RAG Search Directory Read DOCX RAG Search EXA Search Web Loader File Read File Write Firecrawl Crawl Website Firecrawl Scrape Website Firecrawl Search Github Search Hyperbrowser Load Tool Linkup Search Tool LlamaIndex Tool LangChain Tool Google Serper Search S3 Reader Tool S3 Writer Tool Scrapegraph Scrape Tool Scrape Element From Websi

In [26]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry. You can read website content if necessary. "
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [27]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [28]:
embedder_config = {
    "provider": "openai",
    "config": {
        "api_key": os.getenv("OPENAI_API_KEY"),
        "api_base": os.getenv("OPENAI_API_BASE"),
        "model": "text-embedding-3-small",
    }
}

In [29]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True,
  embedder=embedder_config
)

In [30]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 73b9cad2-e7f8-46d3-9c4d-21985a18d6a1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry. You can read website content if necessary. 


🤖 Agent: Senior Support Representative
    Status: In Progress

...

# Agent: Support Quality Assurance Specialist
## Task: Review the response drafted by the Senior Support Representative for DeepLearningAI's inquiry. Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.
Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.
Check for references and sources used to  find the information, ensuring the response is well-supported and leaves no questions unanswered.


# Agent: Support Quality Assurance Specialist
## Final Answer: 
Hi Andrew,

Thanks so much for reaching out with your question about setting up a Crew and adding memory in CrewAI! I’m excited to help you get everything configured smoothly so your Crew can remember past interactions and improve performance.

Here’s a comprehensive overview of how to add memory to your Crew in CrewAI, covering the different types of memory, how to enable and customize them, external p

🚀 Crew: crew
├── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 7122d06e-d653-4225-9ce2-785fc067d3f4
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 7122d06e-d653-4225-9ce2-785fc067d3f4
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7122d06e-d653-4225-9ce2-785fc067d3f4                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 73b9cad2-e7f8-46d3-9c4d-21985a18d6a1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [31]:
from IPython.display import Markdown
Markdown(result.raw)

Hi Andrew,

Thanks so much for reaching out with your question about setting up a Crew and adding memory in CrewAI! I’m excited to help you get everything configured smoothly so your Crew can remember past interactions and improve performance.

Here’s a comprehensive overview of how to add memory to your Crew in CrewAI, covering the different types of memory, how to enable and customize them, external providers, and best practices.

**1. Understanding CrewAI Memory**

CrewAI offers a versatile memory system that allows your agents to remember past interactions, data, and context, which can significantly enhance their capabilities. The main types include:

- **Short-Term Memory:** Temporarily stores recent interactions, often using Retrieval-Augmented Generation (RAG) techniques.
- **Long-Term Memory:** Preserves insights and data across sessions, stored in persistent databases like SQLite.
- **Entity Memory:** Tracks and organizes information about entities (people, places, concepts) encountered during tasks.
- **External Memory:** Integrates with external providers or custom storage solutions for specialized or cross-application memory management.

For detailed info, you can check the [CrewAI Memory Documentation](https://crewai.com/docs/memory).

**2. How to Enable and Configure Memory in Your Crew**

**Basic Activation:**

- When creating your Crew, set the `memory=True` parameter.
- You can customize the memory components by providing specific configurations such as embedding models, storage paths, or external providers.

**Example: Basic Memory Activation**

```python
from crewai import Crew, Process

crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.sequential,
    memory=True,  # Enable default memory system
    verbose=True
)
```

**Example: Using Custom Memory Instances**

For more control, instantiate specific memory modules like FAISS or SQLite:

```python
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage

crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.sequential,
    memory=True,
    long_term_memory=LongTermMemory(
        storage=LTMSQLiteStorage(db_path="./memory.db")
    ),
    short_term_memory=ShortTermMemory(
        storage=RAGStorage(
            embedder_config={
                "provider": "openai",
                "config": {"model": "text-embedding-3-small"}
            },
            type="short_term",
            path="./"
        )
    ),
    entity_memory=EntityMemory(
        storage=RAGStorage(
            embedder_config={
                "provider": "openai",
                "config": {"model": "text-embedding-3-small"}
            },
            type="short_term",
            path="./"
        )
    ),
    verbose=True
)
```

**3. Using External Memory Providers**

- **Mem0:** A popular external provider for scalable, personalized memory. You can set it up with your API key:

```python
import os
from crewai import Crew, Process
from mem0 import MemoryClient

os.environ["MEM0_API_KEY"] = "your-mem0-api-key"

crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.sequential,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"}
    }
)
```

- **Custom Storage:** You can implement your own storage classes if needed, such as FAISS or other vector databases, for tailored solutions.

**4. Additional Tips and Best Practices**

- Use environment variables for sensitive info like API keys and storage paths to keep your setup secure.
- Regularly prune or archive memory to maintain performance.
- Always ensure your memory data is stored securely, respecting privacy and security best practices.
- Customize embedding models and storage options based on your specific use case for optimal results.

**5. Resources and Further Reading**

- [Official CrewAI Memory Documentation](https://crewai.com/docs/memory) — for detailed guides and API references.
- [Tutorials and Examples](https://crewai.com/tutorials) — step-by-step walkthroughs.
- Community forums and support channels for sharing best practices.

If you'd like, I can help you craft a specific code snippet tailored to your project or guide you through integrating a particular memory provider. Just let me know!

Thanks again for reaching out, Andrew. Wishing you and the DeepLearningAI team an awesome experience with CrewAI!

Best regards,  
[Your Name]  
Senior Support Representative, CrewAI

In [32]:
inputs = {
    "customer": "Газпром",
    "person": "Алексей Миллер",
    "inquiry":  "Мне нужна помощь в создании команды (Crew) и её запуске. "
                "Конкретно, интересует, как добавить память (Memory) в мою команду (Crew). "
                "Можете подсказать?"
}

result = crew.kickoff(inputs=inputs)
Markdown(result.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 73b9cad2-e7f8-46d3-9c4d-21985a18d6a1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: Газпром just reached out with a super important ask:
Мне нужна помощь в создании команды (Crew) и её запуске. Конкретно, интересует, как добавить память (Memory) в мою команду (Crew). Можете подсказать?

Алексей Миллер from Газпром is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry. You can read website content if necessary. 


🤖 Agent: Senior Support Representative
    Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 7122d06e-d653-4225-9ce2-785fc067d3f4
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: c6e03eb0-691b-42ed-9d70-4aeebc7f43ca
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
└── 📋 Task: 7122d06e-d653-4225-9ce2-785fc067d3f4
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7122d06e-d653-4225-9ce2-785fc067d3f4                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 73b9cad2-e7f8-46d3-9c4d-21985a18d6a1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Здравствуйте, Алексей!  
Благодарю вас за обращение и интерес к возможностям CrewAI. Ниже я подробно объясню, как создать команду (Crew) с включенной памятью и настроить её под ваши нужды.

1. Создание Crew с активированной памятью  
При инициализации Crew укажите параметр `memory=True`. Это активирует встроенную систему памяти, которая по умолчанию использует SQLite для долговременного хранения и встроенные эмбеддинги для краткосрочной памяти.  
Пример базовой настройки:  
```python
from crewai import Crew, Process

crew = Crew(
    agents=[...],  # ваши агенты
    tasks=[...],   # ваши задачи
    process=Process.sequential,
    memory=True,   # включение памяти
    verbose=True
)
```  

2. Настройка типа памяти и её хранения  
Вы можете дополнительно настроить типы памяти, указав параметры `long_term_memory`, `short_term_memory` и `entity_memory`. Например, для использования SQLite для долговременной памяти и RAG для краткосрочной:  
```python
from crewai.memory import LongTermMemory, ShortTermMemory
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.sequential,
    memory=True,
    long_term_memory=LongTermMemory(
        storage=LTMSQLiteStorage(db_path="./memory.db")
    ),
    short_term_memory=ShortTermMemory(
        storage=RAGStorage(
            embedder_config={
                "provider": "openai",
                "config": {"model": "text-embedding-ada-002"}
            }
        ),
        type="short_term",
        path="./"
    ),
    entity_memory=RAGStorage(
        embedder_config={
            "provider": "openai",
            "config": {"model": "text-embedding-ada-002"}
        },
        type="short_term",
        path="./"
    ),
    verbose=True
)
```  

3. Использование внешней памяти (например, Mem0 или другие системы)  
Можно подключить внешние системы памяти, задав параметры `memory_config`. Например, для Mem0:  
```python
import os
from crewai import Crew, Process

os.environ["MEM0_API_KEY"] = "ваш-ключ-API"

crew = Crew(
    agents=[...],
    tasks=[...],
    process=Process.sequential,
    memory=True,
    memory_config={
        "provider": "mem0",
        "config": {"user_id": "john"}
    },
    verbose=True
)
```  

4. Обновление и сброс памяти  
Для очистки памяти используйте команду CLI:  
```bash
crewai reset-memories --all
```  
или через API:  
```python
crew.reset_memories(command_type='all')
```  

5. Дополнительные рекомендации  
- Используйте переменные окружения для путей и API-ключей для повышения безопасности.  
- Для более сложных сценариев настройте кастомные хранилища или интеграцию с внешними системами.  
- Регулярно проверяйте и обновляйте конфигурацию памяти для оптимальной работы.

Если потребуется помощь с конкретной настройкой или примером кода — обращайтесь, я с радостью помогу!  

Спасибо за доверие и интерес к CrewAI. Желаю вам успешной работы с памятью и реализации ваших проектов!  

С уважением,  
[Ваше имя], старший специалист поддержки  

Для более подробной информации рекомендуем ознакомиться с официальной документацией по памяти CrewAI: [CrewAI Memory Documentation](https://docs.crewai.com/memory).